In [2]:
import cv2
import os
from PIL import Image

In [3]:
cam_url = r"C:\Users\M1061065\OneDrive - Mindtree Limited\Pictures\Camera Roll\WIN_20220104_16_19_32_Pro.mp4"


In [4]:
count = 0
vidcap = cv2.VideoCapture(cam_url)
success,image = vidcap.read()
success = True
fps = vidcap.get(cv2.CAP_PROP_FPS)
frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
duration = frame_count/fps
while success:
    vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))    # added this line 
    success,image = vidcap.read()
    print ('Read a new frame: ', success)
    print(count)
    image = Image.fromarray(image.astype('uint8'), 'RGB')
    image.save("data/frame"+str(count)+".jpg")     # save frame as JPEG file
    count = count + 0.5
    if(count > (duration-0.5)):
        success = False

Read a new frame:  True
0
Read a new frame:  True
0.5
Read a new frame:  True
1.0
Read a new frame:  True
1.5
Read a new frame:  True
2.0
Read a new frame:  True
2.5
Read a new frame:  True
3.0
Read a new frame:  True
3.5
Read a new frame:  True
4.0
Read a new frame:  True
4.5
Read a new frame:  True
5.0
Read a new frame:  True
5.5
Read a new frame:  True
6.0
Read a new frame:  True
6.5
Read a new frame:  True
7.0
Read a new frame:  True
7.5


In [5]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from azure.cognitiveservices.vision.customvision.training.models import ImageFileCreateBatch, ImageFileCreateEntry, Region
from msrest.authentication import ApiKeyCredentials
import os, time, uuid

# Replace with valid values
ENDPOINT = "https://centralindia.api.cognitive.microsoft.com/"
training_key = "c5a175b7f450425ba8df3e4e22c2ec6b"
prediction_key = "e6ecd0955b824244a3d6483e5c7e1097"
prediction_resource_id = "/subscriptions/0d16ec4a-eebf-44fa-8f1d-7803493f8199/resourceGroups/blockchain-group/providers/Microsoft.CognitiveServices/accounts/custom_vision_sk"

credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
prediction_credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(ENDPOINT, prediction_credentials)

In [6]:
publish_iteration_name = "Iteration3"
confidence = 0 
aadhar_card_frame = "none"
project_id = "c61fcd17-ee71-4e68-8ea4-a47b73cdc7f2"


path = r"C:\Users\M1061065\OneDrive - Mindtree Limited\Documents\autenticate_vdo\data"
for image_path in os.listdir(path):

    input_path = os.path.join(path, image_path)
    with open(input_path, mode="rb") as test_data:
        results = predictor.detect_image(project_id, publish_iteration_name, test_data)

    for prediction in results.predictions:
        if((prediction.probability * 100) >= 50) and (prediction.probability * 100) >= confidence :
            confidence = prediction.probability * 100
            aadhar_card_frame = input_path
            break
        
    
        

In [7]:
import asyncio
import io
import glob
import sys
import requests
from urllib.parse import urlparse
from io import BytesIO
# To install this module, run:
# python -m pip install Pillow
from PIL import Image, ImageDraw
from azure.cognitiveservices.vision.face import FaceClient
from msrest.authentication import CognitiveServicesCredentials
from azure.cognitiveservices.vision.face.models import TrainingStatusType, Person

KEY = "471b6c10f789468c8424da69ac81c8dd"

ENDPOINT = "https://proof-verification-sk.cognitiveservices.azure.com/"

face_client = FaceClient(ENDPOINT, CognitiveServicesCredentials(KEY))

In [8]:
if aadhar_card_frame != "none":
    
    image = open(r"C:\Users\M1061065\OneDrive - Mindtree Limited\Documents\autenticate_vdo\data\frame0.5.jpg", 'rb')
    detected_faces1 = face_client.face.detect_with_stream(image, detection_model='detection_03')
    source_image1_id = detected_faces1[0].face_id
    print('{} face(s) detected from image {}.'.format(len(detected_faces1), "selfie" ))
    
    image1 = open(aadhar_card_frame, 'rb')
    detected_faces2 = face_client.face.detect_with_stream(image1, detection_model='detection_03')
    source_image2_id = detected_faces2[0].face_id
    print('{} face(s) detected from image {}.'.format(len(detected_faces2), "aadhar_card_frame"))
    
    verify_result_same = face_client.face.verify_face_to_face(source_image1_id, source_image2_id)
    if verify_result_same.confidence > 0.25:
        print("VERIFIED!")
    else:
        print("MISMATCH")
else:
    print("No aadhar card found!")

1 face(s) detected from image selfie.
1 face(s) detected from image aadhar_card_frame.
VERIFIED!


In [9]:
verify_result_same.confidence

0.27701

In [ ]:
def getRectangle(faceDictionary):
    rect = faceDictionary.face_rectangle
    left = rect.left
    top = rect.top
    right = left + rect.width
    bottom = top + rect.height
    
    return ((left, top), (right, bottom))
url = r"C:\Users\M1061065\OneDrive - Mindtree Limited\Documents\autenticate_vdo\test_img\false_aadhar_3.jpg"
imag = Image.open(url)
print('Drawing rectangle around face... see popup for results.')
draw = ImageDraw.Draw(imag)
for face in detected_faces2:
    draw.rectangle(getRectangle(face), outline='red')
imag

In [ ]:
verify_result_same.confidence 